<a href="https://colab.research.google.com/github/Serena428/Rader_performance_prediction/blob/main/Test4_model_sy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import pandas as pd
import numpy as np
import warnings

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.kernel_ridge import KernelRidge

from sklearn.ensemble import BaggingRegressor

from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import StackingRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error

# Global settings

import plotly.express as px
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


warnings.filterwarnings("ignore") # To ignore warnings
n_jobs = -1 # This parameter conrols the parallel processing. -1 means using all processors.
random_state = 42 # This parameter controls the randomness of the data. Using some int value to get same results everytime this code is run.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df=pd.read_csv('/content/drive/MyDrive/수연/study/radar/train.csv')
#train_df.head()
len(df)

39607

In [5]:
test_df=pd.read_csv('/content/drive/MyDrive/수연/study/radar/test.csv')
#train_df.head()
len(test_df)

39608

In [6]:
df.head()

,ID,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,...,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TRAIN_00001,70.544,103.320,67.47,1,101.892,74.983,29.45,62.38,245.71,...,29.632,16.083,4.276,-25.381,-25.529,-22.769,23.792,-25.470,-25.409,-25.304
1,TRAIN_00002,69.524,103.321,65.17,1,101.944,72.943,28.73,61.23,233.61,...,33.179,16.736,3.229,-26.619,-26.523,-22.574,24.691,-26.253,-26.497,-26.438
2,TRAIN_00003,72.583,103.320,64.07,1,103.153,72.943,28.81,105.77,272.20,...,31.801,17.080,2.839,-26.238,-26.216,-22.169,24.649,-26.285,-26.215,-26.370
3,TRAIN_00004,71.563,103.320,67.57,1,101.971,77.022,28.92,115.21,255.36,...,34.503,17.143,3.144,-25.426,-25.079,-21.765,24.913,-25.254,-25.021,-25.345
4,TRAIN_00005,69.524,103.320,63.57,1,101.981,70.904,29.68,103.38,241.46,...,32.602,17.569,3.138,-25.376,-25.242,-21.072,25.299,-25.072,-25.195,-24.974


In [7]:
df.columns

Index(['ID', 'X_01', 'X_02', 'X_03', 'X_04', 'X_05', 'X_06', 'X_07', 'X_08',
       'X_09', 'X_10', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17',
       'X_18', 'X_19', 'X_20', 'X_21', 'X_22', 'X_23', 'X_24', 'X_25', 'X_26',
       'X_27', 'X_28', 'X_29', 'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35',
       'X_36', 'X_37', 'X_38', 'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44',
       'X_45', 'X_46', 'X_47', 'X_48', 'X_49', 'X_50', 'X_51', 'X_52', 'X_53',
       'X_54', 'X_55', 'X_56', 'Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 'Y_06',
       'Y_07', 'Y_08', 'Y_09', 'Y_10', 'Y_11', 'Y_12', 'Y_13', 'Y_14'],
      dtype='object')

In [8]:
X = df[['X_01', 'X_02', 'X_03', 'X_04', 'X_05', 'X_06', 'X_07', 'X_08',
       'X_09', 'X_10', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17',
       'X_18', 'X_19', 'X_20', 'X_21', 'X_22', 'X_23', 'X_24', 'X_25', 'X_26',
       'X_27', 'X_28', 'X_29', 'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35',
       'X_36', 'X_37', 'X_38', 'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44',
       'X_45', 'X_46', 'X_47', 'X_48', 'X_49', 'X_50', 'X_51', 'X_52', 'X_53',
       'X_54', 'X_55', 'X_56']]

In [9]:
y = df[['Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 'Y_06',
       'Y_07', 'Y_08', 'Y_09', 'Y_10', 'Y_11', 'Y_12', 'Y_13', 'Y_14']]

#Train and Test Data

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.33, random_state=random_state)

print(f'Training set--> X_train shape= {X_train.shape}, y_train shape= {y_train.shape}')
print(f'Holdout set--> X_test shape= {X_test.shape}, y_test shape= {y_test.shape}')

Training set--> X_train shape= (26536, 56), y_train shape= (26536, 14)
Holdout set--> X_test shape= (13071, 56), y_test shape= (13071, 14)


#k

In [15]:
from sklearn.neighbors import KNeighborsRegressor

knr = KNeighborsRegressor()

knr.fit(X_train, y_train)

print(knr.score(X_test, y_test))

KNeighborsRegressor()

In [17]:
from sklearn.metrics import mean_absolute_error

#test set 에 대한 예측을 만듬
test_prediction = knr.predict(X_test)

mae = mean_absolute_error(y_test, test_prediction)
print(mae)

0.7687072712766535


## test1 - k 근접 이웃 : 3

In [19]:
knr.n_neighbors = 3

#모델을 다시 훈련함
knr.fit(X_train, y_train)
print(knr.score(X_train, y_train))

print(knr.score(X_test, y_test))

0.36853820164626117


In [20]:
from sklearn.metrics import mean_absolute_error

#test set 에 대한 예측을 만듬
test_prediction = knr.predict(X_test)

mae = mean_absolute_error(y_test, test_prediction)
print(mae)

0.8066172114932745


## test2 - k 근접 이웃 : 5

In [24]:
knr.n_neighbors = 5

#모델을 다시 훈련함
knr.fit(X_train, y_train)
print(knr.score(X_train, y_train))

print(knr.score(X_test, y_test))

0.2378856452311411
-0.14255788661054405


In [25]:
from sklearn.metrics import mean_absolute_error

#test set 에 대한 예측을 만듬
test_prediction = knr.predict(X_test)

mae = mean_absolute_error(y_test, test_prediction)
print(mae)

0.7687072712766535


## test3 - k 근접 이웃 : 20

In [26]:
knr.n_neighbors = 20

#모델을 다시 훈련함
knr.fit(X_train, y_train)
print(knr.score(X_train, y_train))

print(knr.score(X_test, y_test))

0.07749675083885862
-0.020536720352052888


In [27]:
from sklearn.metrics import mean_absolute_error

#test set 에 대한 예측을 만듬
test_prediction = knr.predict(X_test)

mae = mean_absolute_error(y_test, test_prediction)
print(mae)

0.7267345268150871
